In [16]:
# !pip install --upgrade pip

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import tensorflow as tf
import json
import random

# from scipy.special import entr
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer

In [17]:
CORES = -1
SEED = 567

# FINAL_EXP_RESULTS_PATH="final_experiments_results"
FINAL_EXP_RESULTS_PATH="clip=False_experiments_results"

print(f"\x1b[32mSEED: {SEED}\x1b[0m")

# set up macros
EXP_NUM_0=0
EXP_NUM_1_1=1.1
EXP_NUM_1_2=1.2
EXP_NUM_2_1=2.1
EXP_NUM_2_2=2.2
EXP_NUM_3_1=3.1
EXP_NUM_3_2=3.2
EXP_NUM_4_1=4.1
EXP_NUM_4_2=4.2
EXP_NUM_5_1=5.1
EXP_NUM_5_2=5.2
EXP_NUM_6=6

# to be added to the seed when reseting the np and tf
EXP_NUM_2=2
EXP_NUM_5=5

CUT_OFF_FREQ_RANGE=list(range(1, 50, 4))
EMA_SPAN_RANGE=list(range(1, 50, 4))
SMA_WINSIZE_RANGE=list(range(1, 50, 4))
OVERLAP_EXP_RANGE=list(range(1, 100, 4))


# FOR EXP 2 AND 5 ACROSS NOTEBOOKS
CHOICE_NUM_PAIRS=len(list(range(1, 50, 4)))

WINDOW_SIZE_LST = [125, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]
WINDOW_SIZE_LST_NN = [125, 250, 500, 750, 1000]

OVERLAP=0.5

FILTER_ORDER=10
SAMPLING_FREQ=100
FILTFILT=1

DASH_MACRO_NUM=-99999

SAVE_PATH_NAME="SCNN_trained_models"
CLASSIFIER_TYPE_LST=["OCSVM", "kNN", "IF", "LOF"]
    


def init_experiment_params(exp_config, exp_num=0):
    
    np.random.seed(SEED+exp_num)
    tf.random.set_seed(SEED+exp_num)

    exp_config.SMA_per_win_winsize=None
    exp_config.EMA_per_win_span=None
    exp_config.Butter_per_win_argdict=None
    exp_config.cut_off_freq=None
    exp_config.winsize=None
    exp_config.span=None

    exp_config.filter_order = FILTER_ORDER
    exp_config.sampling_freq = SAMPLING_FREQ
    exp_config.filtfilt = FILTFILT

    print("reseting experiment params successful!")

SEED: 567


In [18]:
NUM_PAIR_LIMIT_TRAIN_1000 = 43693
NUM_PAIR_LIMIT_VALID_1000 = 23527

NUM_PAIR_LIMIT_TRAIN_1500 = 18993
NUM_PAIR_LIMIT_VALID_1500 = 10227

NUM_PAIR_LIMIT_TRAIN_2000 = 10348
NUM_PAIR_LIMIT_VALID_2000 = 5614

In [19]:
# user 47 is assumed to be the last element and take the index of 29 and this affects the code in making windows and preprocessing
# as user 47 has shorter length time series and has to be handled separately
user_ids = [1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 28, 29, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 47]
randomized_data_idx = list(range(len(user_ids)))
random.Random(SEED).shuffle(randomized_data_idx)
# split_idx = 2 * (len(randomized_data_idx)//3) + 1
# train_set = randomized_data_idx[: split_idx]
# test_set = randomized_data_idx[split_idx: ]
# # train_set = randomized_data_idx
# print(f"train_set: {train_set}\ntest_set: {test_set}")
# # train_set = test_set
# # test_set = train_set
# print(f"train_set: {train_set}\ntest_set: {test_set}")

# Split the data into 3 equal parts
n = len(randomized_data_idx)
fold_size = n // 3

THREE_FOLD_CV=[]
for i in range(3):
    test = set(randomized_data_idx[i * fold_size: (i + 1) * fold_size])
    train = set([x for x in randomized_data_idx if x not in test])
    THREE_FOLD_CV.append((train, test))
    
THREE_FOLD_CV

[({0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 14, 15, 16, 18, 19, 22, 23, 24, 25, 28},
  {7, 8, 11, 13, 17, 20, 21, 26, 27, 29}),
 ({0, 2, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 20, 21, 23, 26, 27, 29},
  {1, 3, 5, 6, 16, 19, 22, 24, 25, 28}),
 ({1, 3, 5, 6, 7, 8, 11, 13, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29},
  {0, 2, 4, 9, 10, 12, 14, 15, 18, 23})]

In [33]:
TRAINING_CONFIG_DICT_FILE_NAME_CV_DICT={\
                                        "Butter":
                                        {0: "results_dict_Butter33-cv0-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                         1: "results_dict_Butter33-cv1-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                         2: "results_dict_Butter33-cv2-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                        },
                                        "Butter-EMA":
                                        {0: "results_dict_Butter33-EMA20-cv0-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                         1: "results_dict_Butter33-EMA20-cv1-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                         2: "results_dict_Butter33-EMA20-cv2-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                        },
                                        "Butter-SMA":
                                        {0: "results_dict_Butter33-SMA20-cv0-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                         1: "results_dict_Butter33-SMA20-cv1-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                         2: "results_dict_Butter33-SMA20-cv2-1000-SCNN_3_123_conv_1_dense_arg_dict_default_thesis.json",
                                        },
                                        
}

# TRAINING_CONFIG_DICT_FILE_NAME="model_archi_performance_lr_dict.json"
TRAINING_CONFIG_DICT_FOLDER_PATH="siamese_cnn_results_final"

TRAINING_CONFIG_CV_DICT={"Butter": {},
                         "Butter-EMA": {},
                         "Butter-SMA": {},
                        }

for prep_key in TRAINING_CONFIG_CV_DICT:
    
    for cv_split_idx in range(len(THREE_FOLD_CV)):
        
        training_config_dict_file_name = TRAINING_CONFIG_DICT_FILE_NAME_CV_DICT[prep_key][cv_split_idx]
        # print(training_config_dict_file_name)
        
        with open(f"{TRAINING_CONFIG_DICT_FOLDER_PATH}/{training_config_dict_file_name}", 'r') as file:
            
            training_config_dict=json.load(file)
            
        TRAINING_CONFIG_CV_DICT[prep_key][cv_split_idx]=training_config_dict
        
TRAINING_CONFIG_CV_DICT["Butter"][0]['1000'].keys()

dict_keys(['SCNN_3_1_conv_1_dense_arg_dict_default', 'SCNN_3_2_conv_1_dense_arg_dict_default', 'SCNN_1_3_conv_1_dense_arg_dict_default', 'SCNN_3_123_conv_1_dense_arg_dict_default'])

In [1]:
list(range(1, 50, 4))

[1, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49]

In [16]:
MAGENTA = (202/255, 18/255, 125/255)

In [ ]:
METRIC_WINSIZE_TABLE_COLS = [("Window Size", 125), ("Window Size", 250), ("Window Size", 500), ("Window Size", 750), ("Window Size", 1000), 
                 ("Window Size", 1250), ("Window Size", 1500), ("Window Size", 1750), ("Window Size", 2000), ("", 'mean'), ("", 'variance')]

STYLER_ERR_FORMAT_DICT={key: "{:.4f}" for key in METRIC_WINSIZE_TABLE_COLS}
STYLER_IMPROVEMENT_FORMAT_DICT={key: "{:.2f}" for key in METRIC_WINSIZE_TABLE_COLS}

In [17]:
get_new_scaler_dict = {"StandardScaler": StandardScaler, "MinMaxScaler": MinMaxScaler, "Normalizer": Normalizer, 
                       "MaxAbsScaler": MaxAbsScaler, "RobustScaler": RobustScaler, "PowerTransformer": PowerTransformer}

sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize": 26, "axes.labelsize": 26, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 20, "ytick.labelsize": 20})

err_distro_rc={'figure.figsize':(20,10), 'figure.dpi': 180, "legend.fontsize": 30, "axes.labelsize": 30, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 25, "ytick.labelsize": 25}

In [18]:
sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize": 9, "axes.labelsize": 11, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 11, "ytick.labelsize": 11})
# sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize":11, "axes.labelsize": 11, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
#             "xtick.labelsize": 11, "ytick.labelsize": 11})

sns.set_style("darkgrid")
sns.set_context("paper")
sns.set(font="sans")
sns.set_palette("tab10")

# for plain matplotlib:
plt.style.use(["seaborn-darkgrid", "seaborn-paper"])
plt.rc("font", family="sans", size=8)
plt.rc("axes", titlesize=6)
plt.rc("axes", labelsize=6)
plt.rc("xtick", labelsize=6)
plt.rc("ytick", labelsize=6)
plt.rc("xtick.major", pad=1)
plt.rc("ytick.major", pad=3)
plt.rc("legend", fontsize=6)
plt.rc("figure", titlesize=6)

In [19]:
# Define a style I use a lot for boxplots:
utils_boxplot_style = dict(
    color="tab:blue",
    linewidth=0.5,
    saturation=1,
    width=0.7,
    flierprops=dict(
        marker="o", markersize=2, markerfacecolor="none", markeredgewidth=0.5
    ),
)

# Define a style I use a lot for lineplots:
utils_lineplot_style = dict(
    color="tab:blue", linewidth=0.5, marker="o", markersize=3, markeredgewidth=0.5
)

In [32]:
print(f"FINAL_EXP_RESULTS_PATH: {FINAL_EXP_RESULTS_PATH}")
print(20*'-' + "\x1b[32mSEED and CONSTANTS imported\x1b[0m" + 20*'-')

--------------------SEED and CONSTANTS imported--------------------
